# Building and Training NMF Model Training Encoder

In [1]:
import boto3,os

os.chdir('train-container')

In [2]:
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-nmf-container'
tag = ':nmflatest'
training_repository_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository}'

In [3]:
training_repository_uri

'064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container'

In [97]:
from sagemaker import get_execution_role
from sagemaker.tuner import IntegerParameter, HyperparameterTuner
role = get_execution_role()
role

'arn:aws:iam::064258348567:role/service-role/AmazonSageMaker-ExecutionRole-20200131T104492'

In [87]:
!docker build -t $ecr_repository .

Sending build context to Docker daemon  43.01kB
Step 1/10 : FROM python:3.7-slim-buster
 ---> 30a42f143b4e
Step 2/10 : MAINTAINER ASC "mbod@asc.upenn.edu jmparelman@gmail.com"
 ---> Using cache
 ---> 7e549f0fe312
Step 3/10 : RUN mkdir -p /data/input /data/output
 ---> Using cache
 ---> 08b11b662b04
Step 4/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends         build-essential         python-dev         python3-pip     && pip3 install -U pip
 ---> Using cache
 ---> f8f48d39855c
Step 5/10 : RUN     pip3 install -U pandas          joblib          sklearn
 ---> Using cache
 ---> bb2b2b1b23c8
Step 6/10 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 3217cf1e0586
Step 7/10 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 5fe90e51307f
Step 8/10 : ENV PATH="/opt/ml:${PATH}"
 ---> Using cache
 ---> a1f5ac870916
Step 9/10 : COPY scripts/run_nmf.py /opt/ml/run_nmf.py
 ---> a00cba0fd23f
Step 10/10 : ENTRYPOINT ["python3","/opt/ml/run_nmf.py"]
 ---> Running in

In [88]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 064258348567.dkr.ecr.us-east-1.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [89]:
!aws ecr create-repository --repository-name $ecr_repository



An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-nmf-container' already exists in the registry with id '064258348567'


In [90]:
!docker tag {ecr_repository} "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"
!docker push "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"

The push refers to repository [064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container]

017b0ef3: Preparing 
f015d5b1: Preparing 
ca6bd59c: Preparing 
2ff91b95: Preparing 
10e872c2: Preparing 
6bb9ffea: Preparing 
663e622b: Preparing 
845af46d: Preparing 
017b0ef3: Pushed lready exists kB4Alatest: digest: sha256:e214edb03f4e66863b460f2d53959818ea170df2d91ca8a7b378e6c7cd7e1cbd size: 2211


In [91]:
import sagemaker as sage
from time import gmtime,strftime

sess = sage.Session()

In [93]:
data_location = 's3://ascsagemaker/JMP_congressional_nmf/dtms/110.pkl'
image = "064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nmf-container:latest"

nmf = sage.estimator.Estimator(image,
                              role,1, 'ml.m4.2xlarge',
                              output_path = 's3://ascsagemaker/JMP_congressional_nmf/models',
                              sagemaker_session=sess)

In [103]:
hyperparameter_ranges = {"k":IntegerParameter(20,30)}
objective_metric_name = 'coherence'
metric_definitions = [{'Name': 'coherence',
                       'Regex': 'coherence: ([0-9\\.]+)'}]

In [104]:
tuner = HyperparameterTuner(nmf,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions=metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=3)

In [105]:
tuner.fit(data_location)

....................................................................................................*


UnexpectedStatusException: Error for HyperParameterTuning job sagemaker-nmf-contai-210428-0142: Failed. Reason: No objective metrics found after running 5 training jobs. Please ensure that the custom algorithm is emitting the objective metric as defined by the regular expression provided.